## Imports

In [1]:
import psycopg2
import os
import sys
import requests
import json
import pathlib
import pickle
import hashlib
from os import path
from dotenv import load_dotenv, find_dotenv
from datetime import datetime
from alive_progress import alive_bar

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.data.hash import compute_hash
from src.data.constants import ABS_FILE_PATH_ITEMS
from src.data.database import fetch_and_store_data

In [2]:
compute_hash(ABS_FILE_PATH_ITEMS)

'460ac842e32a71a7b96a80dfbbb5b3413ef6ea39df746ada285fbbc2fcb77f94'

In [2]:
fetch_and_store_data('test_market_data')

Attempting to fetch and insert data...
on 0: Successfully stored data into table for region ID: 0.                                                             
on 1: Successfully stored data into table for region ID: 30000142.                                                      
on 2: Successfully stored data into table for region ID: 30000144.                                                      
on 3: Successfully stored data into table for region ID: 60003760.                                                      
on 4: Successfully stored data into table for region ID: 60008494.                                                      
on 5: Successfully stored data into table for region ID: 60011866.                                                      
on 6: Successfully stored data into table for region ID: 60004588.                                                      
on 7: Successfully stored data into table for region ID: 60005686.                                                

In [39]:

def connect_to_database():
    """ Connects to the postgresql database

    Returns:
        database connection: used to execute sql queries or creating tables.
    """
    
    conn = psycopg2.connect(host=DATABASE_URL,
                            database=DATABASE_NAME,
                            user=DATABASE_USER,
                            password=DATABASE_PASSWORD,
                            port=DATABASE_PORT)
    return conn

def create_table(table_name):
    """ create a table by the name of 'table_name' provided in the PostgreSQL database"""
    command = (
        """
        CREATE TABLE """ + table_name + """ (
            id SERIAL PRIMARY KEY,
            time TIMESTAMP,
            region_id INTEGER,
            data JSON
        )
        """)
    
    conn = connect_to_database()
    try:

        # connect to the PostgreSQL server
        cur = conn.cursor()
        
        # create table one by one
        cur.execute(command)
        # close communication with the PostgreSQL database server
        cur.close()
        # commit the changes
        conn.commit()
        
        print("Table", table_name, "created successfully")
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

## Table of Contents

1. [Data Collection](#data-collection)
2. [Data Validation](#data-validation)
3. [Preprocessing](#preprocessing)

## Data Collection

In [20]:
def compute_hash(filepath):
    """ Hashlib library to compute a hash for a file using sha256 algo.

    Args:
        filepath (string): path to the file in which we want to hash

    Returns:
        string: A hexadecimal string of the hash
    """
    
    # The size of each read from the file (64Kb)
    BLOCK_SIZE = 65536 

    # Create the hash object
    file_hash = hashlib.sha256() 
    
    with open(filepath, 'rb') as f:
        fb = f.read(BLOCK_SIZE)
        
        # While there is still data being read from the file
        while len(fb) > 0: 
            file_hash.update(fb) 
            fb = f.read(BLOCK_SIZE) 

    return file_hash.hexdigest() # Get the hexadecimal digest of the hash
    

def get_raw_material_names():
    """ Return a list of raw material names from from ABS_FILE_PATH_ITEMS

    Returns:
        raw_material_names: item names of interest for forcast
    """
    
    # List that will be returned after it has been populated
    raw_material_names = []
    
    if path.exists(ABS_FILE_PATH_ITEMS):

        file = open(ABS_FILE_PATH_ITEMS, "r")
        
        # Read each line from ABS_FILE_PATH_ITEMS and append each item name into the list
        for item in file.readlines():
            raw_material_names.append(item.strip('\n'))

        file.close()
        
        # Make sure that the item names are not repeated
        raw_material_names = list(set(raw_material_names))
        
        print(raw_material_names)
        return raw_material_names
        
    else:
        print(ABS_FILE_PATH_ITEMS, "does not exist.")

def get_item_id(item_name=None):
    """ Returns the item ID from using the API endpoint https://www.fuzzwork.co.uk/api/typeid.php?typename=Silicon

    Args:
        item_name (string): Name of the raw material. Defaults to None.

    Returns:
        int: ids that are of type int
    """
    
    item_id = None
    
    api_url = "https://www.fuzzwork.co.uk/api/typeid.php?typename=" + item_name
    r = requests.get(api_url)
    item_id = r.json()['typeID']
    
    if item_id:
        return item_id
    
def fetch_data(region_id, item_id):
    """ Returns JSON given an input of the region and item from an API

    Args:
        region_id (int): id that is assigned to each major market region.
        item_id (int, list): id or list of ids that is/are assigned to each raw material.

    Returns:
        dict: JSON Data from https://market.fuzzwork.co.uk/aggregates/?region=30000142&types=9828
    """
    
    # Formating so that it'll be accepted by the API endpoint
    item_id = str(item_id).strip('[]').replace(" ", "")
    
    api_url = "https://market.fuzzwork.co.uk/aggregates/?region=" + str(region_id) + "&types=" + item_id
    r = requests.get(api_url)
    
    # encoding as json
    raw_material_data = r.json()
    
    return raw_material_data

def insert_data(table_name='market_data', region_id=None, json_data=None):
    """ Connects with postgresql database and inserts records into a table.

    Args:
        table_name (str, optional): _description_. Defaults to 'raw_material_data'.
        region_id (int, optional): _description_. Defaults to None.
        json_data (json, optional): _description_. Defaults to None.
    """
    
    time = datetime.now()
    
    try:
        conn = connect_to_database()
        cursor = conn.cursor()
        
        # sql to insert the json into table
        insert_command ="INSERT INTO " + table_name + " (TIME, REGION_ID, DATA) VALUES (%s, %s, %s)"
        
        # Serializing json
        json_object = json.dumps(json_data)
        
        # Inserting into table
        values = (time, region_id, json_object)
        cursor.execute(insert_command, values)
        
        # Close and commit changes to database server
        conn.commit()
        conn.close()
        
        print("Successfully stored data into table for region ID: " + str(region_id) + ".")
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [21]:
# Functions related to serialization

def pickle_data(item, filepath):
    """ Picklizes the 'item' and saves it to a 'filepath'.

    Args:
        item (any): Object to be picklized
        filepath (string): The filepath that you want to save the picklized object to.
    """
    with open(filepath, 'wb') as pickle_file:
        pickle_items = pickle.dump(item, pickle_file)
        
def load_pickle_data(filepath):
    """ Loads a picklized object from a pickle 'filepath'.

    Args:
        filepath (string): filepath to the picklized data

    Returns:
        any: The picklized data that was stored in the filepath
    """
    
    with open(filepath, 'rb') as pickle_file:
        data = pickle.load(pickle_file)
        
    return data

In [14]:
# TODO: Implement concurrency when doing API calls

def fetch_and_store_data(table_name="market_data"):
    """ GET Requests the Eve Online API endpoint https://market.fuzzwork.co.uk/aggregates/?region=30000142&types=9828 
    which takes two params region and types. This JSON data is then stored
    
    Region: The location that the markets are located in
    - There are 7 Regions
        - Global - 0
        - Jita - 30000142
        - Perimeter - 30000144
        - Jita 4-4 CNAP - 60003760
        - Amarr VIII - 60008494
        - Dodixie - 60011866
        - Rens - 60004588
        - Hek - 60005686
        
    Types: The ID of the raw material, List of IDs: https://docs.google.com/spreadsheets/d/1X7mi7j-_yV5lq-Yd2BraE-t4QE_a4IKv2ZuCBSLD6QU/edit?usp=sharing
    """
    
    # Checks if items in items.txt has changed and if it did makes api calls
    old_hash = load_pickle_data(HASH_PATH)
    new_hash = compute_hash(ABS_FILE_PATH_ITEMS)
    
    if old_hash != new_hash:
        items = get_raw_material_names()
        item_ids = [ get_item_id(i) for i in items ]
        pickle_data(item_ids, HASH_PATH)
        
    else:
        item_ids = load_pickle_data(ITEMS_PATH)
        
    

    json_data = {}

    # Fetch the data for each region and raw material id
    print("Attempting to fetch and insert data...")
    with alive_bar(len(REGIONS), force_tty=True) as bar:
        for r in REGIONS:
            
            # Fetching the data
            data = fetch_data(region_id=r, item_id=item_ids)
            
            # Storing the data into a postgresql table
            insert_data(table_name='market_data', region_id=r, json_data=data)
            bar()
    
    print("Successfully stored data for all regions!")
    
    

In [10]:
if __name__ == "__main__":
    fetch_and_store_data()

src/data/items.txt does not exist.
Attempting to fetch and insert data...
on 0: can only concatenate str (not "int") to str                                                                       
on 1: can only concatenate str (not "int") to str                                                                       
on 2: can only concatenate str (not "int") to str                                                                       
on 3: can only concatenate str (not "int") to str                                                                       
on 4: can only concatenate str (not "int") to str                                                                       
on 5: can only concatenate str (not "int") to str                                                                       
on 6: can only concatenate str (not "int") to str                                                                       
on 7: can only concatenate str (not "int") to str                              

## Preprocessing